Order Book Query, Sorting & Preparing.

In [1]:
pip install websocket-client

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Fluffy\AppData\Local\Programs\PythonCodingPack\python.exe -m pip install --upgrade pip' command.


In [2]:
import copy
import json
import ssl
import time
import websocket

In [3]:
class GeminiOrderBook(object):

    def __init__(self):
        self.bids = {}
        self.asks = {}
        self.bids_sorted = []
        self.asks_sorted = []

    def insert(self, price, remaining, side):
        if side == 'bid':
            if remaining == 0:
                if price in self.bids:
                    del self.bids[price]
            else:
                self.bids[price] = remaining
        elif side == 'ask':
            if remaining == 0:
                if price in self.asks:
                    del self.asks[price]
            else:
                self.asks[price] = remaining
        else:
            print('WARNING : UNKNOWN SIDE {}'.format(side))
    
    def sort_bids_asks(self):
        self.bids_sorted = sorted([(price, remaining) for price, remaining in self.bids.items()], reverse = True)
        self.asks_sorted = sorted([(price, remaining) for price, remaining in self.asks.items()])

        #TOP 5 BIDS & ASKS
        self.bids_sorted = self.bids_sorted[: 5]
        self.asks_sorted  = self.asks_sorted[:5]

        self.bids = dict(self.bids_sorted)
        self.asks = dict(self.asks_sorted)

    def get_copy_bids_asks(self):
        return copy.deepcopy(self.bids_sorted), copy.deepcopy(self.asks_sorted)

Market Data Crawler:

In [5]:
class MarketDataCrawler:
    def __init__(self, symbol, output_file):
        self.orderbook = GeminiOrderBook()
        self.output_file = output_file
        print("Connect Gemini")

        self.ws = websocket.WebSocketApp('wss://api.gemini.com/v1/marketdata/{}'.format(symbol)),
                                          on_message = lambda ws, message: self.on_message(message),
                                          on_error = lambda ws, error: self.on_error(ws, error)
        print('Receiving Data BROOOOO')
        while True:
            try:
                self.ws.run_forever(sslop={'cert_reqs': ssl.CERT_NONE})
            except:
                pass
    def on_error(self, ws, error):
        print(error)
    def on_message(self, message):
        data = json.loads(message)
        for event in data['events']:
            price, remaining, side = float(event['price']), float(event['remaining']), event['side']
            self.orderbook.insert(price, remaining, side)
        
        self.orderbook.sort_bids_asks()
        with open(self.output_file, 'a+') as f:
            bids, asks = self.orderbook.get_copy_bids_asks()
            output = {
                'bids': bids,
                'asks': asks,
                'ts': int(time.time() * 1000)
            }
            f.write(json.dumps(output) + '\n')
if __name__ == '__main__':
    crawler =  MarketDataCrawler(symbol = 'BTCUSD', output_file = 'ETH_Orderbook.txt')


IndentationError: unexpected indent (Temp/ipykernel_9000/4249299012.py, line 8)